# Degradation Modelling 

This rover shows how degradation modelling can be performed to model the resilience of an engineered system over its entire lifecycle.

In [1]:
from fmdtools.sim.approach import SampleApproach, NominalApproach
import fmdtools.analyze as an
import fmdtools.sim.propagate as prop
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
import scipy.stats as stats


import multiprocessing as mp

In [2]:
prop

<module 'fmdtools.sim.propagate' from 'c:\\users\\dhulse\\documents\\github\\fmdtools\\fmdtools\\sim\\propagate.py'>

In [3]:
from rover_model_human import *
from rover_model import plot_trajectories, plot_map

NameError: name 'FxnBlock' is not defined

In [ ]:
"""
import importlib
import rover_model_human
importlib.reload(rover_model_human)
importlib.reload(an.plot)
from rover_model_human import *
import fmdtools.analyze as an
"""

Degradation models are defined independently of the fault model, but have attributes (e.g., functions) which may correspond to it directly.

Because degradation may only occur in specific functions/flows (and may not have inter-functional dependencies), it is not necessary for the degradation model to have the same 

In [ ]:
deg_mdl = RoverDegradation()

In [ ]:
figax = an.graph.show(deg_mdl, gtype='fxngraph')

In [ ]:
deg_mdl_hum_long = HumanDegradationLong()

In [ ]:
figax = an.graph.show(deg_mdl_hum_long, gtype='fxngraph')

In [ ]:
deg_mdl_hum_short = HumanDegradationShort()
figax = an.graph.show(deg_mdl_hum_short, gtype='fxngraph')

In [ ]:
fault_mdl = Rover()
fig, ax = an.graph.show(fault_mdl, gtype='fxnflowgraph')

In [ ]:
fig.savefig("func_model.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

In [ ]:
mdl = Rover(params=gen_params('sine', amplitude=1))
endresults, mdlhist = prop.nominal(mdl)
plot_map(mdl, mdlhist)
fig = plt.gcf()

In [ ]:
fig.savefig("sine_rover_environment.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

As shown, there are two degradation models here:
- one which focusses solely on faults in the drive system, and
- one which focusses on the human degradation of fatigue
Below we simulate these to model to the degradation behaviors being modelled in this drive system.

## Drive Degradation

In [ ]:
deg_mdl = RoverDegradation()
endresults, mdlhist = prop.nominal(deg_mdl)
fig = an.plot.mdlhists(mdlhist)

The major behaviors are:
- wear
- corrosion
- friction
- drift

These behaviors result from the accumulation of the following rates over each time-step:
- yaw_load 
- corrode_rate
- wear_rate

These degradation behaviors have additionally been defined to simulate stochastically if desired:

In [ ]:
deg_mdl = RoverDegradation()
endresults_deg, mdlhist_deg = prop.nominal(deg_mdl, run_stochastic=True)
fig = an.plot.mdlhists(mdlhist_deg)

To get averages/percentages over a number of scenarios, we can view these behaviors over a given number of random seeds:

In [ ]:
nomapp_deg = NominalApproach()
nomapp_deg.add_seed_replicates('test', 100)
endclasses_deg, mdlhists_deg = prop.nominal_approach(deg_mdl, nomapp_deg, run_stochastic=True)

In [ ]:
fig, ax = an.plot.mdlhists(mdlhists_deg, aggregation='mean_std', fxnflowvals={'Drive':['friction','drift']}, title="", xlabel='lifecycle time (months)')

In [ ]:
fig.savefig("drive_degradations.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

As shown, while wear and friction proceed monotonically, drift can go one way or another, meaning that whether the rover drifts left or right is basically up to chance. We can further look at slices of these distributions:

In [ ]:
fig, axs = an.plot.metric_dist_from(mdlhists_deg, [10, 20, 25], fxnflowvals='Drive', bins=10, alpha=0.5)

In [ ]:
fig, axs = an.plot.metric_dist_from(mdlhists_deg, [1, 10, 20, 25], fxnflowvals='Drive', bins=10, alpha=0.5)

Given the parameter information (friction and drift) that the degradation model produced, we can now simulate the model with this information over time in the nominal scenarios.

In [ ]:
behave_nomapp = NominalApproach()
behave_nomapp.add_param_ranges(gen_sample_params_comp, 'behave_nomapp', mdlhists_deg, t=(1,12,1), scen = (1,30,1), turn='sine')

In [ ]:
mdl=Rover(valparams={'end_rad':3.0})
behave_endclasses, behave_mdlhists = prop.nominal_approach(mdl, behave_nomapp)

In [ ]:
fig = an.plot.nominal_vals_2d(behave_nomapp, behave_endclasses, 't', 'scen', xlabel='lifecycle time (months)', ylabel='scenario', legendloc='upper left', metric='at_finish', nomlabel='True')

In [ ]:
fig.savefig("drive_deg_envelope.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

In [ ]:
fig = plot_trajectories(behave_mdlhists)

As shown, as the time (and thus degradation) increases, the rover becomes more likely to be unable to complete the mission. In this case, it results in the rover not completing the mission in time.

In [ ]:
behave_endclasses_nested, behave_mdlhists_nested, behave_apps_nested = prop.nested_approach(mdl, behave_nomapp, pool=mp.Pool(5), faults='Drive', track='flows')


Finally, we can also visualize simulate and then view the effect of the degradation on average resilience...

In [ ]:
res_comp = an.tabulate.resilience_factor_comparison(behave_nomapp, behave_endclasses_nested, ['t'], 'at_finish', percent=False)
fig = an.plot.resilience_factor_comparison(res_comp, stack=True, maxy=1.1, xlabel='lifecycle time (months)', figsize=(5.5,4), title="", stat="proportion complete")   

In [ ]:
fig.savefig("drive_resilience_degradation.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

As shown, while there is some resilience early in the lifecycle (resulting in a small proportion of faults being recovered), this resilience goes away with degradation.

In [ ]:
res_comp

## Human Degradation

We can also perform this assessment for the human error model, which is split up into two parts:
- long term "degradation" of experience over months
- short term "degradation" of stress and fatigue over a day

In [ ]:
deg_mdl_hum_long = HumanDegradationLong()
endresults,  mdlhist_hum_long = prop.nominal(deg_mdl_hum_long)
fig,ax = an.plot.mdlhists(mdlhist_hum_long)

In [ ]:
nomapp_hum_long = NominalApproach()
experience_param = np.random.default_rng(seed=101).gamma(1,1.9,101)
experience_param = list(experience_param)
nomapp_hum_long.add_param_ranges(gen_long_degPSF_param, 'nomapp_hum_long', experience_param, scen = (0,25,1))

In [ ]:
endclasses, mdlhists_hum_long = prop.nominal_approach(deg_mdl_hum_long, nomapp_hum_long, run_stochastic=True)

In [ ]:
fig,ax = an.plot.mdlhists(mdlhists_hum_long, aggregation='mean_std', time_slice = [1,5,9,13], legend_loc=-1, xlabel="time (months)", ylabels={("functions", "Control", "experience"):"experience"})

In [ ]:
fig.savefig("experience_degradation.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

Short-term degradation

In [ ]:
deg_mdl_hum_short = HumanDegradationShort()
endresults, mdlhist_hum_short = prop.nominal(deg_mdl_hum_short)
fig, axs = an.plot.mdlhists(mdlhist_hum_short)
endresults, mdlhist_hum_short = prop.nominal(deg_mdl_hum_short, run_stochastic=True)
fig, axs = an.plot.mdlhists(mdlhist_hum_short)

In [ ]:
mdlhist_hum_short

short-term degradation (over no external params)

In [ ]:
nomapp_human_short = NominalApproach()
nomapp_human_short.add_seed_replicates('test', 25)

In [ ]:
endclasses_human_short, mdlhists_human_short = prop.nominal_approach(deg_mdl_hum_short, nomapp_human_short, run_stochastic=True)

In [ ]:
fig, ax = an.plot.mdlhists(mdlhists_human_short, aggregation='mean_std')

short-term degradation over long-term params

In [ ]:
stress_param = np.random.default_rng(seed=101).gamma(2,1.9,101)
stress_param = list(stress_param)

In [ ]:
nomapp_short_long = NominalApproach()
nomapp_short_long.add_param_ranges(gen_short_degPSF_param, 'nomapp', mdlhists_hum_long, stress_param, scen = (0,25,1), t= (1,15,4))
nomapp_short_long.update_factor_seeds('nomapp', 'scen')

In [ ]:
endclasses, mdlhists_hum_short_long = prop.nominal_approach(deg_mdl_hum_short, nomapp_short_long, run_stochastic=True)    

In [ ]:
nomapp_short_long.ranges['nomapp']['inputranges']

In [ ]:
comp_groups = nomapp_short_long.get_param_scens('nomapp', 't')

In [ ]:
fig, ax = an.plot.mdlhists(mdlhists_hum_short_long, fxnflowvals={"Control":"stress"}, comp_groups=comp_groups, aggregation='mean_std',
                           xlabel="time (hours)", cols=1,ylabels={("functions", "Control", "stress"):"stress"}, legend_title='month')

In [ ]:
fig.savefig("stress_degradation.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

In [ ]:
behave_nomapp_hum = NominalApproach()
behave_nomapp_hum.add_param_ranges(gen_human_params_combined, 'behave_nomapp_hum', mdlhists_hum_short_long, nomapp_short_long, 'nomapp', t_stress=(1,11, 2), t_exp=(1,15,4), scen = (1,25, 1), turn='sine')

In [ ]:
mdl=Rover(valparams={'end_rad':3.0})
behave_endclasses_hum, behave_mdlhists_hum = prop.nominal_approach(mdl, behave_nomapp_hum, run_stochastic=True)   

In [ ]:
fig = an.plot.nominal_vals_2d(behave_nomapp_hum, behave_endclasses_hum, 't_stress', 'scen', xlabel='single-day time (hours)', ylabel='scenario',  metric='at_finish', nomlabel='True')

In [ ]:
fig = an.plot.nominal_vals_2d(behave_nomapp_hum, behave_endclasses_hum, 't_exp', 'scen', xlabel='learning time (months)', ylabel='scenario',  metric='at_finish', nomlabel='True')

In [ ]:
fig = an.plot.nominal_vals_3d(behave_nomapp_hum, behave_endclasses_hum, 't_stress', 't_exp', 'scen', xlabel='single-day time (hours)', ylabel = 'experience time (months)', zlabel='scenario',  metric='at_finish', nomlabel='True', title="Mission Successes")

In [ ]:
fig.savefig("human_nominal_joint_degradation.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

In [ ]:
fig = plot_trajectories(behave_mdlhists_hum)

In [ ]:
fig, ax = an.plot.mdlhists(behave_mdlhists_hum, fxnflowvals='GlobalPSF',   legend_loc=False)

In [ ]:
fig, ax = an.plot.metric_dist_from(behave_mdlhists_hum, 1, fxnflowvals={'GlobalPSF':'stress'})

In [ ]:
nom_comp_hum = an.tabulate.nominal_factor_comparison(behave_nomapp_hum, behave_endclasses_hum, ['t_exp','t_stress'], ['at_finish'], give_ci=True, return_anyway=True, percent=True)
nom_comp_hum

In [ ]:
scendict = behave_nomapp_hum.get_param_scens('behave_nomapp_hum', 't_exp','t_stress')
late_scens = scendict[13,5]
early_scens= scendict[13,1]

In [ ]:
late_scens

In [ ]:
early_scens

In [ ]:
behave_nomapp_hum.scenarios['behave_nomapp_hum_90']['properties']['params']

In [ ]:
plt.hist([behave_nomapp_hum.scenarios[scen]['properties']['params']['stress'] for scen in early_scens], alpha=0.5, label='early')
plt.hist([behave_nomapp_hum.scenarios[scen]['properties']['params']['stress'] for scen in late_scens], alpha=0.5, label='late')
plt.legend()

In [ ]:
behave_endclasses_hum

In [ ]:
plt.hist([int(behave_endclasses_hum[scen]['at_finish']) for scen in early_scens], alpha=0.5, label='early')
plt.hist([float(behave_endclasses_hum[scen]['at_finish']) for scen in late_scens], alpha=0.5, label='late')
plt.legend()

In [ ]:
plt.hist([behave_endclasses_hum[scen]['end_dist'] for scen in early_scens], alpha=0.5, label='early')
plt.hist([behave_endclasses_hum[scen]['end_dist'] for scen in late_scens], alpha=0.5, label='late')
plt.legend()

In [ ]:
fig = an.plot.nominal_factor_comparison(nom_comp_hum, 'at_finish', maxy=1.1, xlabel='single-day time (hours)', figsize=(10,6), title="", error_bars=True)  

In [ ]:
nom_comp_hum = an.tabulate.nominal_factor_comparison(behave_nomapp_hum, behave_endclasses_hum, ['t_exp'], ['at_finish'], percent=False)
nom_comp_hum

In [ ]:
nom_comp_hum = an.tabulate.nominal_factor_comparison(behave_nomapp_hum, behave_endclasses_hum, ['t_stress'], ['at_finish'],  percent=False)
nom_comp_hum

As for the human resilience:

In [ ]:
behave_endclasses_nested_hum, behave_mdlhists_nested_hum, behave_apps_nested_hum = prop.nested_approach(mdl, behave_nomapp_hum, pool=mp.Pool(5), faults='Drive', track='flows')

In [ ]:
res_comp_hum = an.tabulate.resilience_factor_comparison(behave_nomapp_hum, behave_endclasses_nested_hum, ['t_exp','t_stress'], 'at_finish')
res_comp_hum

In [ ]:
fig = an.plot.resilience_factor_comparison(res_comp_hum, stack=True, maxy=1.1, xlabel='t_experience (months), t_stress (hours)', figsize=(10,4), title="", stat="proportion complete")  

In [ ]:
fig.savefig("human_resilience_degradation.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

In [ ]:
res_comp_hum

## Combined Degradation

Idea: make same resilience plots as before but with degradation of resilience to drive faults at 0, 4, and 8 hours of fatigue.

Need method of doing this:
- take params for one and replace with the other (kind of hacky and requires independence)
- create composite of both? (how do you determine the merge between output params?)
- use a combined method with both histories as input and sample over t_life and t_day (use this one)

In [ ]:
joint_nomapp = NominalApproach()
joint_nomapp.add_param_ranges(gen_sample_params_combined, 'behave_jointapp', mdlhists_deg, mdlhists_hum_short_long, nomapp_short_long, stress_id='nomapp',  t_comp=(1,26,12), t_stress=(1,11, 4), t_exp=(1,10,8), scen = (1,25,1), turn='sine')

In [ ]:
mdl=Rover(valparams={'end_rad':3.0})
behave_endclasses_joint, behave_mdlhists_joint = prop.nominal_approach(mdl, joint_nomapp)

In [ ]:
fig = an.plot.nominal_vals_3d(joint_nomapp, behave_endclasses_joint, 't_comp', 't_stress', 'scen', xlabel='lifecycle time (months)', ylabel='single-day time (hours)', zlabel='scenario',  metric='at_finish', nomlabel='True')

In [ ]:
nom_comp_hum = an.tabulate.nominal_factor_comparison(joint_nomapp, behave_endclasses_joint, ['t_exp', 't_comp', 't_stress'], ['at_finish'], percent=False, return_anyway=True)
nom_comp_hum

In [ ]:
nom_comp_hum.columns

In [ ]:
fig = an.plot.nominal_factor_comparison(nom_comp_hum, 'at_finish', maxy=1.1, xlabel='single-day time (hours)', figsize=(16,4), title="", error_bars=True)  

Resilience assessment

In [ ]:
behave_endclasses_nested_joint, behave_mdlhists_nested_hum, _ = prop.nested_approach(mdl, joint_nomapp, pool=mp.Pool(5), faults='Drive', track='flows')

In [ ]:
res_joint = an.tabulate.resilience_factor_comparison(joint_nomapp, behave_endclasses_nested_joint, ['t_exp', 't_comp', 't_stress'], 'at_finish', percent=False) 

In [ ]:
res_joint

In [ ]:
fig = an.plot.resilience_factor_comparison(res_joint, stack=True, maxy=1.1, xlabel='t_experience, t_component, t_stress', figsize=(12,4), title="", stat="proportion complete") 

In [ ]:
fig.savefig("joint_resilience_degradation.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)